In [2]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas

In [3]:
import pandas as pd

# Define the path to the Excel file
file_path_sentek = '/Users/yaniksitta/co2_dashboard/24 KSU TAPS Sentek.xlsx'

# Load the Excel file
excel_file_sentek = pd.ExcelFile(file_path_sentek)

# Initialize an empty list to hold the data from each worksheet
all_sheets_sentek = []

# Iterate over each sheet in the Excel file
for sheet_name_sentek in excel_file_sentek.sheet_names:
    # Read the sheet, skipping the first two rows
    sheet_data_sentek = pd.read_excel(file_path_sentek, sheet_name=sheet_name_sentek, skiprows=2)
    
    # Remove the " C" from the column names if present
    sheet_data_sentek.columns = [col.replace(" C", "")
                                     .replace("Temp Sensor #6 (55 cm)", "Temp. Sensor #6 (55 cm)")
                                 for col in sheet_data_sentek.columns]
    
    # Add a new column with the sheet name (Team Number)
    sheet_data_sentek['Team Number'] = sheet_name_sentek
    
    # Append the sheet data to the list
    all_sheets_sentek.append(sheet_data_sentek)

# Combine all sheets into a single DataFrame
combined_sentek = pd.concat(all_sheets_sentek, ignore_index=True)

# Convert the 'Timestamp' column to datetime format and specify the desired format
combined_sentek['Timestamp'] = pd.to_datetime(combined_sentek['Timestamp'])

# Remove all rows where column V1 value is -1
combined_sentek = combined_sentek[combined_sentek["V1"] != -1]

# Round the 'Timestamp' to the nearest hour
combined_sentek['Timestamp'] = combined_sentek['Timestamp'].dt.round('H')

# Show the combined DataFrame
combined_sentek


,Timestamp,V1,V2,Sensor #1 (5 cm),Interpolated Sum Sensor #1 (5 cm),Temp. Sensor #1 (5 cm),Sensor #2 (15 cm),Interpolated Sum Sensor #2 (15 cm),Temp. Sensor #2 (15 cm),Sensor #3 (25 cm),...,Sensor #7 (65 cm),Interpolated Sum Sensor #7 (65 cm),Temp. Sensor #7 (65 cm),Sensor #8 (75 cm),Interpolated Sum Sensor #8 (75 cm),Temp. Sensor #8 (75 cm),Sensor #9 (85 cm),Interpolated Sum Sensor #9 (85 cm),Temp. Sensor #9 (85 cm),Team Number
0,2024-05-27 19:00:00,13.953,-1.000,39.00141,1348.5660,21.62000,53.94933,1876.5730,21.20999,50.72508,...,37.69760,307.7849,16.41000,36.36094,295.5445,15.79999,31.88238,107.28780,15.37000,Team #3 Data
1,2024-05-27 20:00:00,13.996,-1.000,39.01892,1423.5510,20.82001,53.98945,2087.7360,21.03000,50.73800,...,37.73787,353.9729,16.31000,36.39152,324.3752,15.79999,31.97125,122.55290,15.16000,Team #3 Data
2,2024-05-27 20:00:00,13.970,-1.000,39.82043,1049.1840,20.22000,53.94933,1527.6480,21.00000,50.74445,...,37.79161,287.3690,16.20001,36.42977,276.5934,15.76999,32.00683,99.42731,15.16000,Team #3 Data
3,2024-05-27 20:00:00,13.992,-1.000,39.90023,1066.5130,19.62000,53.97340,1527.4610,20.82001,50.75737,...,37.80505,287.2325,16.20001,36.50634,276.6559,15.64999,32.03650,99.54457,15.16000,Team #3 Data
4,2024-05-27 21:00:00,14.012,13.666,39.74958,1037.9170,19.22000,53.99747,1527.2690,20.62000,50.76383,...,37.81850,287.0995,16.20001,36.52167,279.4164,15.70001,32.06025,100.14580,15.16000,Team #3 Data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56516,2024-09-17 07:00:00,13.891,-1.000,22.22545,148.5871,20.31000,28.48298,181.0686,21.35001,35.39825,...,38.13113,1022.9340,19.92001,40.06640,1212.7190,19.26999,41.04163,3987.70700,19.57999,Team #34 Data
56517,2024-09-17 08:00:00,13.921,-1.000,22.19212,149.8248,20.29001,28.50482,179.1859,21.19000,35.42100,...,38.13834,1018.6730,19.89001,40.07381,1221.5770,19.26999,41.04163,3971.17600,19.57999,Team #34 Data
56518,2024-09-17 08:00:00,13.949,-1.000,22.20322,150.2241,20.13000,28.48298,179.0820,21.19000,35.42100,...,38.13834,1014.5150,19.89001,40.07381,1221.5770,19.26999,41.05574,3972.32700,19.54999,Team #34 Data
56519,2024-09-17 09:00:00,13.881,-1.000,22.18101,144.6304,20.07999,28.46660,179.4992,21.14001,35.40963,...,38.14555,1014.4300,19.87000,40.05159,1221.6290,19.29999,41.04163,3971.17600,19.57999,Team #34 Data


In [4]:
import pandas as pd

# Define the path to the Excel file for Mesonet data
file_path_mesonet = 'Mesonet_Monthly_Colby.xlsx'

# Load the Excel file
excel_file_mesonet = pd.ExcelFile(file_path_mesonet)

# Initialize an empty list to hold the data from each worksheet
all_sheets_mesonet = []

# Iterate over each sheet in the Excel file
for sheet_name_mesonet in excel_file_mesonet.sheet_names:
    # Read the sheet, skipping the first two rows
    sheet_data_mesonet = pd.read_excel(file_path_mesonet, sheet_name=sheet_name_mesonet)
    
    # Add a new column with the sheet name (Month)
    sheet_data_mesonet['Month'] = sheet_name_mesonet
    
    # Append the sheet data to the list
    all_sheets_mesonet.append(sheet_data_mesonet)

# Combine all sheets into a single DataFrame
combined_mesonet = pd.concat(all_sheets_mesonet, ignore_index=True)

# Drop rows 2 and 3 (which correspond to indices 1 and 2)
combined_mesonet = combined_mesonet.drop([0, 1], axis=0)

# Remove duplicate Timestamps
combined_mesonet = combined_mesonet.drop_duplicates(subset='Timestamp')

# Drop 'Colby' and 'Month' columns
combined_mesonet = combined_mesonet.drop(columns=['Station', 'Month'], errors='ignore')

# Convert the Timestamp column to datetime in combined_mesonet
combined_mesonet['Timestamp'] = pd.to_datetime(combined_mesonet['Timestamp'])

# Convert the specified columns to float
float_columns = ['AirTemperature', 'Precipitation', 'WindDirection2m', 'WindSpeed2m',
                 'SoilTemperature5cm', 'RelativeHumidity','SoilTemperature10cm', 'SolarRadiation']

# Use pd.to_numeric to convert these columns to float, with error handling
for col in float_columns:
    combined_mesonet[col] = pd.to_numeric(combined_mesonet[col], errors='coerce')

# Round the Timestamp to the nearest minute in combined_mesonet
combined_mesonet['Timestamp'] = combined_mesonet['Timestamp'].dt.round('1min')

# Format the Timestamp to "Year-Month-Day Hours:Rounded Minutes" in combined_mesonet
combined_mesonet['Timestamp'] = combined_mesonet['Timestamp'].dt.strftime('%Y-%m-%d %H:%M')


# Show the combined DataFrame
combined_mesonet


,Timestamp,AirTemperature,RelativeHumidity,Precipitation,WindDirection2m,WindSpeed2m,SoilTemperature5cm,SoilTemperature10cm,SolarRadiation
2,2024-04-01 00:00,5.8,92.2,0.0,89.7,4.6,11.1,11.0,0.0
3,2024-04-01 01:00,6.2,89.2,0.0,60.9,3.6,10.8,10.8,0.0
4,2024-04-01 02:00,5.9,88.1,0.0,41.9,4.3,10.5,10.5,0.0
5,2024-04-01 03:00,5.4,89.0,0.0,13.9,5.0,10.3,10.3,0.0
6,2024-04-01 04:00,4.3,95.5,0.0,0.5,4.9,10.1,10.1,0.0
...,...,...,...,...,...,...,...,...,...
4543,2024-10-01 19:00,16.1,25.4,0.0,164.6,2.3,18.9,18.9,10.0
4544,2024-10-01 20:00,13.9,30.5,0.0,158.7,2.4,18.5,18.8,0.0
4545,2024-10-01 21:00,12.2,35.2,0.0,163.9,3.0,18.0,18.5,0.0
4546,2024-10-01 22:00,10.9,38.9,0.0,175.4,3.4,17.6,18.3,0.0


In [5]:
combined_sentek['Timestamp'] = pd.to_datetime(combined_sentek['Timestamp'])

# Round the Timestamp to the nearest minute in combined_sentek
combined_sentek['Timestamp'] = combined_sentek['Timestamp'].dt.round('1min')

# Format the Timestamp to "Year-Month-Day Hours:Rounded Minutes" in combined_sentek
combined_sentek['Timestamp'] = combined_sentek['Timestamp'].dt.strftime('%Y-%m-%d %H:%M')

# Merge the two datasets using 'Timestamp' from combined_sentek as the basis (left join)
merged_data_weather = pd.merge(combined_sentek, combined_mesonet, on='Timestamp', how='left')

# Interpolate to fill in the NaN values in combined_mesonet columns
# Interpolation will linearly estimate values between known points for numerical columns
mesonet_columns = combined_mesonet.columns.drop('Timestamp')  # All columns except 'Timestamp'
merged_data_weather[mesonet_columns] = merged_data_weather[mesonet_columns].interpolate(method='linear')

# Show the merged DataFrame with interpolated values
merged_data_weather

,Timestamp,V1,V2,Sensor #1 (5 cm),Interpolated Sum Sensor #1 (5 cm),Temp. Sensor #1 (5 cm),Sensor #2 (15 cm),Interpolated Sum Sensor #2 (15 cm),Temp. Sensor #2 (15 cm),Sensor #3 (25 cm),...,Temp. Sensor #9 (85 cm),Team Number,AirTemperature,RelativeHumidity,Precipitation,WindDirection2m,WindSpeed2m,SoilTemperature5cm,SoilTemperature10cm,SolarRadiation
0,2024-05-27 19:00,13.953,-1.000,39.00141,1348.5660,21.62000,53.94933,1876.5730,21.20999,50.72508,...,15.37000,Team #3 Data,24.7,28.7,0.0,58.1,3.1,23.6,22.7,218.7
1,2024-05-27 20:00,13.996,-1.000,39.01892,1423.5510,20.82001,53.98945,2087.7360,21.03000,50.73800,...,15.16000,Team #3 Data,23.1,31.5,0.0,86.7,2.5,22.9,22.3,45.3
2,2024-05-27 20:00,13.970,-1.000,39.82043,1049.1840,20.22000,53.94933,1527.6480,21.00000,50.74445,...,15.16000,Team #3 Data,23.1,31.5,0.0,86.7,2.5,22.9,22.3,45.3
3,2024-05-27 20:00,13.992,-1.000,39.90023,1066.5130,19.62000,53.97340,1527.4610,20.82001,50.75737,...,15.16000,Team #3 Data,23.1,31.5,0.0,86.7,2.5,22.9,22.3,45.3
4,2024-05-27 21:00,14.012,13.666,39.74958,1037.9170,19.22000,53.99747,1527.2690,20.62000,50.76383,...,15.16000,Team #3 Data,20.6,37.6,0.0,90.7,2.1,22.1,21.9,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56491,2024-09-17 07:00,13.891,-1.000,22.22545,148.5871,20.31000,28.48298,181.0686,21.35001,35.39825,...,19.57999,Team #34 Data,17.3,92.4,0.0,162.5,5.1,21.6,21.9,12.2
56492,2024-09-17 08:00,13.921,-1.000,22.19212,149.8248,20.29001,28.50482,179.1859,21.19000,35.42100,...,19.57999,Team #34 Data,18.6,87.8,0.0,171.1,5.8,21.5,21.8,135.8
56493,2024-09-17 08:00,13.949,-1.000,22.20322,150.2241,20.13000,28.48298,179.0820,21.19000,35.42100,...,19.54999,Team #34 Data,18.6,87.8,0.0,171.1,5.8,21.5,21.8,135.8
56494,2024-09-17 09:00,13.881,-1.000,22.18101,144.6304,20.07999,28.46660,179.4992,21.14001,35.40963,...,19.57999,Team #34 Data,21.4,72.8,0.0,181.5,6.7,21.5,21.8,325.0


## Graph #1

In [7]:
import pandas as pd
import plotly.graph_objects as go
import panel as pn

# Enable Panel extension
pn.extension()

# Remove duplicate timestamps for the same team from combined_sentek
combined_sentek_cleaned = combined_sentek.sort_values('Timestamp').drop_duplicates(subset=['Timestamp', 'Team Number'], keep='first')

# Remove duplicate timestamps in combined_mesonet
combined_mesonet_cleaned = combined_mesonet.sort_values('Timestamp').drop_duplicates(subset=['Timestamp'], keep='first')

# Merge the Sentek and Mesonet dataframes on the 'Timestamp' column
merged_data_temp = pd.merge(combined_sentek_cleaned, combined_mesonet_cleaned[['Timestamp', 'AirTemperature']], on='Timestamp', how='inner')

# Create a list of unique team numbers
team_numbers_temp = merged_data_temp['Team Number'].unique()

# Function to update the graph based on the selected team
def update_temperature_graph(selected_team):
    # Create an empty figure
    fig_temp = go.Figure()

    # Filter data for the selected team
    filtered_data = merged_data_temp[merged_data_temp['Team Number'] == selected_team]

    # Add traces for Temp. Sensor #1, #2, #3, and AirTemperature for the selected team
    fig_temp.add_trace(go.Scatter(x=filtered_data['Timestamp'], y=filtered_data['AirTemperature'], 
                                  mode='lines', line=dict(color='cadetblue'), 
                                  name=f'Air Temperature - Team {selected_team}', visible=True))    
    fig_temp.add_trace(go.Scatter(x=filtered_data['Timestamp'], y=filtered_data['Temp. Sensor #1 (5 cm)'], 
                                  mode='lines', line=dict(color='blue'), 
                                  name=f'Temp. Sensor #1 (5 cm) - Team {selected_team}', visible=True))

    # Add upper limit line (32 degrees Celsius)
    fig_temp.add_trace(go.Scatter(x=filtered_data['Timestamp'], y=[32] * len(filtered_data),
                                  mode='lines', line=dict(color='#d62728', dash='dot'),
                                  name='Upper Temp. Limit for ideal Growth (32°C)', visible=True))

    # Add lower limit line (15 degrees Celsius)
    fig_temp.add_trace(go.Scatter(x=filtered_data['Timestamp'], y=[15] * len(filtered_data),
                                  mode='lines', line=dict(color='#d62728', dash='dot'),
                                  name='Lower Temp. Limit for ideal Growth (15°C)', visible=True))

    # Update the layout for the entire chart
    fig_temp.update_layout(
        title=f"Team {selected_team} Temperature Sensors and Air Temperature",
        xaxis_title="Timestamp",
        yaxis_title="Temperature (°C)",
        hovermode="x unified",
        legend=dict(
            orientation="h",  # Horizontal legend
            y=-0.3,           # Position the legend below the plot
            x=0.5,
            xanchor='center'
    )
)
    return fig_temp

# Panel widgets
team_selector_temp = pn.widgets.Select(name='Team Selector for Temperature Dashboard', options=team_numbers_temp.tolist(), value=team_numbers_temp[0])

# Panel plot
@pn.depends(team_selector_temp.param.value)
def plot_temperature(selected_team):
    return pn.panel(update_temperature_graph(selected_team))

# Layout the dashboard
dashboard_weather_2 = pn.Column(
    "# Temperature Sensors vs. Air Temperature Dashboard",
    team_selector_temp,
    plot_temperature
)

# Show the dashboard
dashboard_weather_2.servable()



pn.extension('plotly')



Column
    [0] Markdown(str)
    [1] Select(name='Team Selector f..., options=['Team #3 Data', ...], value='Team #3 Data')
    [2] ParamFunction(function, _pane=Plotly, defer_load=False)

### Graph #2  Mesonet Weather Data


In [9]:
import pandas as pd
import panel as pn
import datetime

# Convert Timestamp to datetime
combined_mesonet['Timestamp'] = pd.to_datetime(combined_mesonet['Timestamp'])

# Get the minimum and maximum dates from the Timestamp column
min_date = combined_mesonet['Timestamp'].min().date()
max_date = combined_mesonet['Timestamp'].max().date()

# Panel extension
pn.extension()

# Date selector widget for daily and weekly information
date_picker = pn.widgets.DatePicker(name='Select Date', value=min_date, start=min_date, end=max_date, linked=True)

# Buttons to move to next or previous day or week
next_day_button = pn.widgets.Button(name='Next Day', button_type='primary')
next_week_button = pn.widgets.Button(name='Next Week', button_type='primary')
previous_day_button = pn.widgets.Button(name='Previous Day', button_type='primary')
previous_week_button = pn.widgets.Button(name='Previous Week', button_type='primary')

# Function to increment or decrement the selected date by one day or one week
def update_date(event):
    if event.obj.name == 'Next Day':
        new_date = date_picker.value + datetime.timedelta(days=1)
    elif event.obj.name == 'Next Week':
        new_date = date_picker.value + datetime.timedelta(weeks=1)
    elif event.obj.name == 'Previous Day':
        new_date = date_picker.value - datetime.timedelta(days=1)
    elif event.obj.name == 'Previous Week':
        new_date = date_picker.value - datetime.timedelta(weeks=1)

    if min_date <= new_date <= max_date:
        date_picker.value = new_date

# Attach the update function to the buttons
next_day_button.on_click(update_date)
next_week_button.on_click(update_date)
previous_day_button.on_click(update_date)
previous_week_button.on_click(update_date)

# Function to update the daily summary based on selected date
def update_daily_summary(selected_date):
    # Filter data for the selected date
    filtered_data = combined_mesonet[combined_mesonet['Timestamp'].dt.date == selected_date]
    
    if filtered_data.empty:
        return pn.pane.Alert("No data available for the selected date.", alert_type="warning")
    
    # Calculate summary metrics
    avg_temp = filtered_data['AirTemperature'].mean()
    min_temp = filtered_data['AirTemperature'].min()
    max_temp = filtered_data['AirTemperature'].max()
    avg_humidity = filtered_data['RelativeHumidity'].mean()
    total_precipitation = filtered_data['Precipitation'].sum()
    
    # Create a summary panel with right-aligned text
    summary = pn.Column(
        pn.pane.Markdown(f"### Summary for {selected_date}"),
        pn.Row(pn.pane.Markdown("**Average Temperature:**", width=150), pn.pane.Markdown(f"{avg_temp:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Min Temperature:**", width=150), pn.pane.Markdown(f"{min_temp:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Max Temperature:**", width=150), pn.pane.Markdown(f"{max_temp:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Average Humidity:**", width=150), pn.pane.Markdown(f"{avg_humidity:.2f}%", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Total Precipitation:**", width=150), pn.pane.Markdown(f"{total_precipitation:.2f} mm", styles={'text-align': 'right'})),
        css_classes=['summary-panel'],
        styles={'background': '#f0f0f5'},
        margin=(10, 15),
        padding=(10, 15),
    )
    return summary

# Interactive panel to update daily view
@pn.depends(date_picker)
def daily_view(date_picker):
    return update_daily_summary(date_picker)

# Function to update the weekly summary based on selected week ending date
def update_weekly_summary(ending_date):
    start_date = ending_date - datetime.timedelta(days=6)
    
    # Filter data for the selected week
    filtered_data = combined_mesonet[(combined_mesonet['Timestamp'].dt.date >= start_date) &
                                     (combined_mesonet['Timestamp'].dt.date <= ending_date)]
    
    if filtered_data.empty:
        return pn.pane.Alert("No data available for the selected week.", alert_type="warning")
    
    # Calculate summary metrics
    avg_temp = filtered_data['AirTemperature'].mean()
    min_temp = filtered_data['AirTemperature'].min()
    max_temp = filtered_data['AirTemperature'].max()
    avg_humidity = filtered_data['RelativeHumidity'].mean()
    total_precipitation = filtered_data['Precipitation'].sum()
    
    # Create a summary panel with right-aligned text
    summary = pn.Column(
        pn.pane.Markdown(f"### Summary for Week\n### **{start_date}** to **{ending_date}**"),
        pn.Row(pn.pane.Markdown("**Average Temperature:**", width=150), pn.pane.Markdown(f"{avg_temp:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Min Temperature:**", width=150), pn.pane.Markdown(f"{min_temp:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Max Temperature:**", width=150), pn.pane.Markdown(f"{max_temp:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Average Humidity:**", width=150), pn.pane.Markdown(f"{avg_humidity:.2f}%", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Total Precipitation:**", width=150), pn.pane.Markdown(f"{total_precipitation:.2f} mm", styles={'text-align': 'right'})),
        css_classes=['summary-panel'],
        styles={'background': '#f0f0f5'},
        margin=(10, 15),
        padding=(10, 15),
    )
    return summary

# Interactive panel to update weekly view
@pn.depends(date_picker)
def weekly_view(date_picker):
    return update_weekly_summary(date_picker)

# Create the dashboard layout with tabs
dashboard_weather_1 = pn.Tabs(
    ("Daily Information", pn.Column(
        pn.pane.Markdown("# Daily Data Summary", styles={'font-size': '20px', 'font-weight': 'bold'}),
        date_picker,
        pn.Row(previous_day_button, next_day_button),
        daily_view
    )),
    ("Weekly Information", pn.Column(
        pn.pane.Markdown("# Weekly Data Summary", styles={'font-size': '20px', 'font-weight': 'bold'}),
        date_picker,
        pn.Row(previous_week_button, next_week_button),
        weekly_view
    ))
)

# Serve the dashboard
dashboard_weather_1.servable()


Tabs
    [0] Column
        [0] Markdown(str, styles={'font-size': '20px', ...})
        [1] DatePicker(end=datetime.date(2024, ..., name='Select Date', start=datetime.date(2024, 4, 1), value=datetime.date(2024, 4, 1))
        [2] Row
            [0] Button(button_type='primary', name='Previous Day')
            [1] Button(button_type='primary', name='Next Day')
        [3] ParamFunction(function, _pane=Column, defer_load=False)
    [1] Column
        [0] Markdown(str, styles={'font-size': '20px', ...})
        [1] DatePicker(end=datetime.date(2024, ..., name='Select Date', start=datetime.date(2024, 4, 1), value=datetime.date(2024, 4, 1))
        [2] Row
            [0] Button(button_type='primary', name='Previous Week')
            [1] Button(button_type='primary', name='Next Week')
        [3] ParamFunction(function, _pane=Column, defer_load=False)

In [10]:
import pandas as pd

# Load the Excel file into a dataframe
file_path = "Forecast_weather_gov.xlsx"
forecast_df = pd.read_excel(file_path, header=None)

# Transpose the dataframe to switch rows to columns
forecast_transposed = forecast_df.transpose()

# Set the first row as the header
forecast_transposed.columns = forecast_transposed.iloc[0]
forecast_transposed = forecast_transposed[1:]

# Convert "Date" column to datetime
forecast_transposed['Date'] = pd.to_datetime(forecast_transposed['Date'])

# Add the hour information to the "Date" column
forecast_transposed['Hour (CDT)'] = forecast_transposed['Hour (CDT)'].astype(int)
forecast_transposed['Datetime'] = forecast_transposed.apply(lambda row: row['Date'] + pd.Timedelta(hours=row['Hour (CDT)']), axis=1)

# Drop the original "Date" and "Hour (CDT)" columns if no longer needed
forecast_transposed.drop(columns=['Date', 'Hour (CDT)'], inplace=True)

# Rename "Datetime" column to "Date" if needed
forecast_transposed.rename(columns={'Datetime': 'Date'}, inplace=True)

# Convert temperature columns from Fahrenheit to Celsius and rename them
def fahrenheit_to_celsius(fahrenheit):
    return round((fahrenheit - 32) * 5.0/9.0, 1)

# Convert and rename relevant columns
if 'Temperature (°F)' in forecast_transposed.columns:
    forecast_transposed['Temperature (°C)'] = forecast_transposed['Temperature (°F)'].astype(float).apply(fahrenheit_to_celsius)
    forecast_transposed.drop(columns=['Temperature (°F)'], inplace=True)

if 'Dewpoint (°F)' in forecast_transposed.columns:
    forecast_transposed['Dewpoint (°C)'] = forecast_transposed['Dewpoint (°F)'].astype(float).apply(fahrenheit_to_celsius)
    forecast_transposed.drop(columns=['Dewpoint (°F)'], inplace=True)

if 'Wind Chill (°F)' in forecast_transposed.columns:
    forecast_transposed['Wind Chill (°C)'] = forecast_transposed['Wind Chill (°F)'].astype(float).apply(fahrenheit_to_celsius)
    forecast_transposed.drop(columns=['Wind Chill (°F)'], inplace=True)

# Display the updated dataframe
forecast_transposed


,Surface Wind (mph),Wind Dir,Gust,Sky Cover (%),Precipitation Potential (%),Relative Humidity (%),Rain,Thunder,Snow,Freezing Rain,Sleet,Date,Temperature (°C),Dewpoint (°C),Wind Chill (°C)
1,5,E,NaN,30,0,76,--,--,--,--,--,2024-11-02 00:00:00,3.9,0.0,2.2
2,5,E,NaN,27,6,79,--,--,--,--,--,2024-11-02 01:00:00,3.9,0.6,2.2
3,5,E,NaN,30,6,81,--,--,--,--,--,2024-11-02 02:00:00,3.3,0.6,1.7
4,5,E,NaN,34,6,90,--,--,--,--,--,2024-11-02 03:00:00,3.3,1.7,1.1
5,5,E,NaN,37,6,92,--,--,--,--,--,2024-11-02 04:00:00,2.8,1.7,0.6
6,5,E,NaN,45,6,98,--,--,--,--,--,2024-11-02 05:00:00,3.3,2.8,1.1
7,6,E,NaN,52,9,100,--,--,--,--,--,2024-11-02 06:00:00,3.3,3.3,1.1
8,5,SE,NaN,71,22,100,SChc,--,--,--,--,2024-11-02 07:00:00,3.9,3.9,1.7
9,5,S,NaN,74,22,100,SChc,--,--,--,--,2024-11-02 08:00:00,5.0,5.0,3.3
10,5,S,NaN,77,22,100,SChc,--,--,--,--,2024-11-02 09:00:00,6.1,6.1,4.4


In [11]:
import pandas as pd
import panel as pn
import datetime


# Convert Date to datetime
forecast_transposed['Date'] = pd.to_datetime(forecast_transposed['Date'])

# Get the minimum and maximum dates from the Date column
min_date_2 = forecast_transposed['Date'].min().date()
max_date_2 = forecast_transposed['Date'].max().date()

# Panel extension
pn.extension()

# Date selector widgets for daily information (made unchangeable)
date_picker_day_1 = pn.widgets.DatePicker(name='Select Date Day 1', value=min_date_2, start=min_date_2, end=max_date_2, disabled=True)
date_picker_day_2 = pn.widgets.DatePicker(name='Select Date Day 2', value=min_date_2 + datetime.timedelta(days=1), start=min_date_2, end=max_date_2, disabled=True)

# Function to update the daily summary based on selected date
def update_daily_summary_2(selected_date_2):
    # Filter data for the selected date
    filtered_data_2 = forecast_transposed[forecast_transposed['Date'].dt.date == selected_date_2]
    
    if filtered_data_2.empty:
        return pn.pane.Alert("No data available for the selected date.", alert_type="warning")
    
    # Calculate summary metrics
    avg_temp_2 = filtered_data_2['Temperature (°C)'].mean()
    min_temp_2 = filtered_data_2['Temperature (°C)'].min()
    max_temp_2 = filtered_data_2['Temperature (°C)'].max()
    avg_humidity_2 = filtered_data_2['Relative Humidity (%)'].mean()
    total_precipitation_2 = filtered_data_2['Precipitation Potential (%)'].mean()
    
    # Create a summary panel with right-aligned text
    summary_2 = pn.Column(
        pn.pane.Markdown(f"### Summary for {selected_date_2}"),
        pn.Row(pn.pane.Markdown("**Average Temperature:**", width=150), pn.pane.Markdown(f"{avg_temp_2:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Min Temperature:**", width=150), pn.pane.Markdown(f"{min_temp_2:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Max Temperature:**", width=150), pn.pane.Markdown(f"{max_temp_2:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Average Humidity:**", width=150), pn.pane.Markdown(f"{avg_humidity_2:.2f}%", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Average Precipitation Potential:**", width=150), pn.pane.Markdown(f"{total_precipitation_2:.2f}%", styles={'text-align': 'right'})),
        css_classes=['summary-panel'],
        styles={'background': '#f0f0f5'},
        margin=(10, 15),
        padding=(10, 15),
    )
    return summary_2

# Create daily summaries for the first and second day
summary_day_1 = update_daily_summary_2(min_date_2)
summary_day_2 = update_daily_summary_2(min_date_2 + datetime.timedelta(days=1))

# Create dashboard tabs for Day 1 and Day 2
dashboard_forecast = pn.Tabs(
    ("Day 1", pn.Column(
        pn.pane.Markdown("# Day 1 Forecast", styles={'font-size': '20px', 'font-weight': 'bold'}),
        date_picker_day_1,
        summary_day_1
    )),
    ("Day 2", pn.Column(
        pn.pane.Markdown("# Day 2 Forecast", styles={'font-size': '20px', 'font-weight': 'bold'}),
        date_picker_day_2,
        summary_day_2
    ))
)

# Serve the dashboard
dashboard_forecast.servable()


Tabs
    [0] Column
        [0] Markdown(str, styles={'font-size': '20px', ...})
        [1] DatePicker(disabled=True, end=datetime.date(2024, ..., name='Select Date Day 1', start=datetime.date(2024, ..., value=datetime.date(2024, ...)
        [2] Column(css_classes=['summary-panel'], margin=(10, 15), styles={'background': '#f0f0f5'})
            [0] Markdown(str)
            [1] Row
                [0] Markdown(str, width=150)
                [1] Markdown(str, styles={'text-align': 'right'})
            [2] Row
                [0] Markdown(str, width=150)
                [1] Markdown(str, styles={'text-align': 'right'})
            [3] Row
                [0] Markdown(str, width=150)
                [1] Markdown(str, styles={'text-align': 'right'})
            [4] Row
                [0] Markdown(str, width=150)
                [1] Markdown(str, styles={'text-align': 'right'})
            [5] Row
                [0] Markdown(str, width=150)
                [1] Markdown(str, styles={'text-align': 'right'})
    [1] Column
        [0] Markdown(str, styles={'font-size': '20px', ...})
        [1] DatePicker(disabled=True, end=datetime.date(2024, ..., name='Select Date Day 2', start=datetime.date(2024, ..., value=datetime.date(2024, ...)
        [2] Column(css_classes=['summary-panel'], margin=(10, 15), styles={'background': '#f0f0f5'})
            [0] Markdown(str)
            [1] Row
                [0] Markdown(str, width=150)
                [1] Markdown(str, styles={'text-align': 'right'})
            [2] Row
                [0] Markdown(str, width=150)
                [1] Markdown(str, styles={'text-align': 'right'})
            [3] Row
                [0] Markdown(str, width=150)
                [1] Markdown(str, styles={'text-align': 'right'})
            [4] Row
                [0] Markdown(str, width=150)
                [1] Markdown(str, styles={'text-align': 'right'})
            [5] Row
                [0] Markdown(str, width=150)
                [1] Markdown(str, styles={'text-align': 'right'})